In [1]:
import os.path
import sys

import numpy as np
import scipy.sparse as sp

from numpy.linalg import norm

from IPython.display import HTML

from arnoldi.decomposition import ArnoldiDecomposition, RitzDecomposition

In [2]:
import arnoldi
root = os.path.join(os.path.dirname(arnoldi.__file__), "..", "..")
sys.path.insert(0, root)
from tests.test_matrices import mark

## Some basic utils

In [3]:
def min_distance(a, b):
    # Compute distance between a and b, ignoring differences due to sign
    # difference of conjugate
    m = np.inf
    for left in (a, np.conj(a)):
        for right in (b, np.conj(b)):
            m = min(m, np.abs(left - right))
            m = min(m, np.abs(left + right))

    return m


def fortran_d_format(x):
    if x == 0:
        return "0.000D+00"
    else:
        exponent = int(np.floor(np.log10(abs(x))))
        mantissa = x / (10 ** exponent)
        # Normalize mantissa to [0.1, 1.0)
        mantissa /= 10
        exponent += 1
        return f"{mantissa:.3f}D{exponent:+03d}"

F = fortran_d_format

## Simple arnoldi decomposition, no restart

In [4]:
def arnoldi_simple(a, k=None):
    n = a.shape[0]
    nev = 1 # Naive arnoldi w/o restart only really works for 1 eigenvalue
    k = k or min(max(2 * nev + 1), n)

    arnoldi = ArnoldiDecomposition(n, k)
    arnoldi.initialize()

    n_iter = arnoldi.iterate(a)

    ritz = RitzDecomposition.from_arnoldi(arnoldi, nev, n_iter)
    return ritz.values, ritz.vectors

In [5]:
A = mark(10)
N = A.shape[0]
NEV = 1

r_values = np.sort(sp.linalg.eigs(A, NEV)[0])[::-1]
print(r_values)

[-1.+0.j]


In [6]:
def display_fancy_table(data, caption=None, headers=None):
    if headers is None:
        headers = ["$m$", "$\\Re(\\lambda)$", "$\\Im(\\lambda)$", "Res. Norm"]
    html = '<table style="border-collapse: collapse; font-family: monospace; text-align: right;">'
    if caption:
        html += f'<caption style="caption-side: bottom; text-align: center; padding-top: 6px; font-style: italic;">{caption}</caption>'
    html += '<thead><tr>' + ''.join(f'<th style="border: 1px solid black; padding: 4px;">{h}</th>' for h in headers) + '</tr></thead>'
    html += '<tbody>'
    for row in data:
        m, re, im, res = row
        html += '<tr>'
        html += f'<td style="border: 1px solid black; padding: 4px;">{m}</td>'
        html += f'<td style="border: 1px solid black; padding: 4px;">{re:.10f}</td>'
        html += f'<td style="border: 1px solid black; padding: 4px;">{im:.1f}</td>'
        html += f'<td style="border: 1px solid black; padding: 4px;">{F(res)}</td>'
        html += '</tr>'
    html += '</tbody></table>'
    return HTML(html)

data = []
for k in [5, 10, 15, 20, 25, 30]:
    ritz_vals, ritz_vecs = arnoldi_simple(A, k)
    data.append([k, np.real(ritz_vals[0]), np.imag(ritz_vals[0]), min_distance(ritz_vals[0], r_values[0])])

display_fancy_table(data, "Convergence as a function of Arnoldi decomposition dimension (reproduction of table 6.1)")

## Simple arnoldi decomposition with naive restarts

In [7]:
def arnoldi_with_naive_restart(a, k, max_iters):
    n = a.shape[0]
    nev = 1 # Arnoldi w/ naive restart only really works for one eigen value
    arnoldi = ArnoldiDecomposition(n, k)

    v = np.random.randn(n)
    v /= norm(v)

    residuals_history = {}

    for i in range(max_iters + 1):
        if i > 0:
            arnoldi.q.fill(0)
            arnoldi.h.fill(0)
        arnoldi.initialize(v)

        n_iter = arnoldi.iterate(a)

        ritz = RitzDecomposition.from_arnoldi(arnoldi, nev, k=n_iter)
        if np.abs(ritz.approximate_residuals[0]) < arnoldi.atol:
            break

        v = ritz.vectors[:, 0]
        v /= norm(v)

        residuals_history[i] = ritz.approximate_residuals

    return ritz.values, ritz.vectors, residuals_history, i

In [8]:
data = []
k = 5
for max_restarts in [10 // k, 20 // k, 30 // k, 40 // k, 50 // k]:
    ritz_vals, ritz_vecs, history, _ = arnoldi_with_naive_restart(A, k, max_restarts)
    data.append([max_restarts * k, np.real(ritz_vals[0]), np.imag(ritz_vals[0]), min_distance(ritz_vals[0], r_values[0])])

display_fancy_table(
    data,
    "Convergence as a function of #restarts, for a fixed Arnoldi decomposition dimension (reproduction of table 6.2)",
    headers=["$restarts$", "$\\Re(\\lambda)$", "$\\Im(\\lambda)$", "Res. Norm"],
)

In [9]:
k = 5
max_iters = 5000
ritz_vals, ritz_vecs, history, n_iter = arnoldi_with_naive_restart(A, k, max_iters)
print(f"Took {n_iter} iterations and ~{n_iter * k} mat vecs to converge")
print(f"residual        {min_distance(ritz_vals[0], r_values[0]):.5e}")

Took 15 iterations and ~75 mat vecs to converge
residual        2.52325e-11


## Arnoldi with explicit restarts and deflation/locking

Deflation means reducing the influence of directions associated to already converged ritz values. One way is to lock the converged ritz vectors when restarting arnoldi decomposition, i.e. removing them from the arnoldi decomposition so that they are not updated anymore. The algorithm below implements such a scheme for the case of finding largest amplitude eigen values.

A basic implementation of deflated iterative Arnoldi, as described in section 6.4 of Numerical methods for large eigenvalue problems, 2nd edition by Yusef Saad:

### Deflated iterative Arnoldi

**A. Start**: Choose an initial vector $v_1$ of norm unity. Set $k := 1$.

**B. Eigenvalue loop**:

1. Arnoldi Iteration. For $j = k, k+1, \dots, m$ do:
   - Compute $w := Av_j$.
   - Compute a set of $j$ coefficients $h_{i,j}$ so that  
     $$
     w := w - \sum_{i=1}^{j} h_{i,j} v_i
     $$
     is orthogonal to all previous $v_i$’s, $i = 1, 2, \dots, j$.
   - Compute  
     $$
     h_{j+1,j} = \|w\|_2, \quad v_{j+1} = \frac{w}{h_{j+1,j}}
     $$

2. Compute approximate eigenvector of $A$ associated with the eigenvalue $\tilde{\lambda}_k$ and its associated residual norm estimate $\rho_k$.

3. Orthonormalize this eigenvector against all previous $v_j$’s to get the approximate Schur vector $\tilde{u}_k$ and define  
   $$
   v_k := \tilde{u}_k
   $$

4. If $\rho_k$ is small enough then (accept eigenvalue):
   - Compute  
     $$
     h_{i,k} = (Av_k, v_i), \quad i = 1, \dots, k
     $$
   - Set $k := k + 1$
   - If $k \ge \texttt{nev}$ then stop, else go to B.1

5. Else go to B.1

